In [1]:
# Import Required Packages
%matplotlib inline
import numpy as np
#import networkx as nx
import time
import random
import json
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from sympy import *
from tqdm.notebook import tqdm

#From Qiskit library
from qiskit import *
from qiskit.visualization import *
from qiskit import IBMQ
from qiskit import BasicAer
from qiskit import Aer, execute
from qiskit.tools.visualization import plot_histogram, plot_state_city
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToQubo
from qiskit.quantum_info import Statevector
from qiskit.providers.aer import StatevectorSimulator, QasmSimulator
from qiskit.circuit import Parameter
from qiskit.optimization.converters import IsingToQuadraticProgram
from qiskit.aqua.components.optimizers import NELDER_MEAD, CRS, ADAM

In [2]:
#Function: Load JSON file
def loadjson( filename ):
    loaded_file = open(filename, "r")
    json_data = json.load(loaded_file)
    loaded_file.close()
    return json_data

#Function: Load Pickle file
def loadpkl( filename ):
    loaded_file = open(filename, "rb")
    pkl_data = pickle.load( loaded_file )
    loaded_file.close()
    return pkl_data

#Function: Get single term coefficients (to 4 decimal places) from Sympy Expression 
def get_coeff( variable, expression ):
    poly = Poly(expression.coeff(variable), domain = 'RR')
    return round(float(poly.coeffs()[len(poly.coeffs())-1]), 4)

#Function: Get coefficient of quadratic term (to 4 decimal places) from Sympy Expression
def quad_coeff( var_1, var_2, expression ):
    return round(float(expression.coeff(var_1).coeff(var_2)), 4)

In [3]:
#Read and store files into variables
car_routes = loadjson( "3cars3routes.json" )
# H = loadjson( "Hamiltonian9qbits.json")
# [objective, Ham, objective_normed, Ham_normed] = loadpkl("expressions.pkl")
[total_cost, constraints] = loadpkl("cost_constraints.pkl")

Q = IndexedBase('Q')
car_routes_variables = {}
for route in car_routes:
    car_routes_variables[route] = Q[route[3],route[10]]
car_routes_variables

no_routes = 3
no_cars = 3
no_qubits = no_routes * no_cars

#Associate binary variable to each Q11, Q12, etc with one index for QuadraticProgram class
binary_vars = {}
for variable in car_routes_variables:
    binary_vars[car_routes_variables[variable]] = 'X' + str(no_routes*(int(variable[3])-1)+int(variable[10]))
Qvariables = tuple(binary_vars.keys())
Xvariables = tuple(binary_vars.values())

#Construct Quadratic Problem using known (quadratic) objective function
penalty = 132*1.2
qubo1 = QuadraticProgram()
linear = []
for variable in car_routes_variables.values():
    qubo1.binary_var( binary_vars[variable] )
    linear.append( get_coeff(variable, total_cost) )
quadratic = {}
for i in range(no_qubits):
    for j in range(i+1, no_qubits):
        pairing = ( Xvariables[i], Xvariables[j] )
        coeff = quad_coeff( Qvariables[i], Qvariables[j], total_cost )
        if int(coeff) != 0:
            quadratic[pairing] = coeff
qubo1.minimize(linear = linear, quadratic = quadratic)

#Add constraints
for car in constraints:
    i = 3*(int(car[3])-1) #to specify index of variable on next line
    qubo1.linear_constraint(linear={'X'+str(i+1): 1, 'X'+str(i+2): 1, 'X'+str(i+3): 1}, sense='==', rhs=1, name=str(car))

#Convert constrained Quadratic Problem into QUBO
converter = QuadraticProgramToQubo(penalty = penalty)
qubo1 = converter.convert(qubo1)
op, offset = qubo1.to_ising()

coeffs = [abs(op.oplist[i].coeff) for i in range(len(op.oplist))]
norm_factor = np.amax(coeffs)

#Normalised objective function
qubo = QuadraticProgram()
linear = []
for variable in car_routes_variables.values():
    qubo.binary_var( binary_vars[variable] )
    linear.append( (get_coeff(variable, total_cost)/norm_factor) )
quadratic = {}
for i in range(no_qubits):
    for j in range(i+1, no_qubits):
        pairing = ( Xvariables[i], Xvariables[j] )
        coeff = quad_coeff( Qvariables[i], Qvariables[j], total_cost )
        if int(coeff) != 0:
            quadratic[pairing] = (coeff/norm_factor)
qubo.minimize(linear = linear, quadratic = quadratic)

#Add constraints
for car in constraints:
    i = 3*(int(car[3])-1) #to specify index of variable on next line
    qubo.linear_constraint(linear={'X'+str(i+1): 1, 'X'+str(i+2): 1, 'X'+str(i+3): 1}, sense='==', rhs=1, name=str(car))

#Convert constrained Quadratic Problem into QUBO
converter = QuadraticProgramToQubo(penalty = penalty/norm_factor)
qubo = converter.convert(qubo)

#Finally obtain the Ising Hamiltonian from QUBO
op, offset = qubo.to_ising()

In [4]:
#Ising Hamiltonian obtained from above
print(op)

SummedOp([
  -0.9041022908897177 * IIIIIIIIZ,
  -0.6670218433670753 * IIIIIIIZI,
  -0.7682472029834843 * IIIIIIZII,
  -0.9041022908897177 * IIIIIZIII,
  -0.7362812999467236 * IIIIZIIII,
  -1.0 * IIIZIIIII,
  -0.9041022908897176 * IIZIIIIII,
  -0.9413958444326054 * IZIIIIIII,
  -0.9573787959509856 * ZIIIIIIII,
  0.42194992008524246 * IIIIIIIZZ,
  0.42194992008524246 * IIIIIIZIZ,
  0.42194992008524246 * IIIIIIZZI,
  0.07991475759190197 * IIIIIZIIZ,
  0.005327650506126798 * IIIIZIIZI,
  0.03196590303676079 * IIIIZIZII,
  0.42194992008524246 * IIIIZZIII,
  0.053276505061267986 * IIIZIIIIZ,
  0.4752264251465104 * IIIZIZIII,
  0.42194992008524246 * IIIZZIIII,
  0.07991475759190197 * IIZIIIIIZ,
  0.07991475759190197 * IIZIIZIII,
  0.053276505061267986 * IIZZIIIII,
  0.055940330314331384 * IZIIIIIIZ,
  0.03196590303676079 * IZIIIIIZI,
  0.055940330314331384 * IZIIIZIII,
  0.045285029302077784 * IZIZIIIII,
  0.47789025039957383 * IZZIIIIII,
  0.053276505061267986 * ZIIIIIIIZ,
  0.00532765050612

In [14]:
#Binary string x of length 9, using cost/obj function from QUBO on QuadraticProgram qp.
def eval_cost(x, qubo_problem):
    return qubo_problem.objective.evaluate([int(x[no_qubits-1-i]) for i in range(no_qubits)])

#Function to optimize the qaoa circuit given a qubo problem
def optimize_qubo(qubo_problem, p, freq_optimal_params_L, freq_optimal_params_B, no_perturb):
    # Inputs: QAOA problem, no_layers, u_v_L, u_v_B, no_perturb
    
    simulator = StatevectorSimulator(method = "statevector")
    optimizer = NELDER_MEAD(maxiter = 10000)
    no_routes = 3
    no_cars = 3
    no_qubits = no_routes * no_cars
    perturb_scale_factor = 0.6
    op, offset = qubo_problem.to_ising()
    generator = np.random.default_rng()
    variable_bounds = [ (-np.pi/2, np.pi/2) for _ in range(2*p)]
    if len(freq_optimal_params_L) != len(freq_optimal_params_B) and len(freq_optimal_params_B) != (p-1)*2:
        print('Check number of input parameters are correct')
        return None
    
    #Construct circuit
    print(f'\nStep {p}: Building circuit...')
    circuit = construct_QAOAcirc(p, op)
    
    #New initial points generated from previous initial points
    params = np.zeros(shape = (2*p,), dtype=float)+0.05  
    initial_freq_params_L = params.copy()
    initial_freq_params_B_0 = params.copy()
    initial_freq_params_L[0:p-1] = freq_optimal_params_L[0:p-1].copy()
    initial_freq_params_L[p:2*p-1] = freq_optimal_params_L[p-1:2*p-2].copy()
    initial_freq_params_B_0[0:p-1] = freq_optimal_params_B[0:p-1].copy()
    initial_freq_params_B_0[p:2*p-1] = freq_optimal_params_B[p-1:2*p-2].copy()
    
    #Generate perturbed initial points     
    if p != 1:
        initial_freq_params_B = np.empty(shape = (no_perturb+1, ), dtype = object)
        initial_freq_params_B[0] = initial_freq_params_B_0.copy()
        u_random = np.array([np.array(generator.normal(0, freq_optimal_params_B[i]**2, no_perturb+1)) for i in range(0,p-1)])
        v_random = np.array([np.array(generator.normal(0, freq_optimal_params_B[i]**2, no_perturb+1)) for i in range(p-1,2*p-2)])
        m = len(initial_freq_params_B)
        for j in range(1,m):
            perturbed_params = params.copy()
            perturbed_params[0:p-1] = initial_freq_params_B_0[0:p-1].copy() + perturb_scale_factor*u_random[:,j].copy()
            perturbed_params[p:2*p-1] = initial_freq_params_B_0[p:2*p-1].copy() + perturb_scale_factor*v_random[:,j].copy()
            initial_freq_params_B[j] = perturbed_params.copy()
        
    print(f'\nStep {p}: Now optimising parameters (Nelder-Mead)...')
    ##Define obj_function that inputs only parameters
    obj_expectation = lambda x: expectation( circuit, qubo_problem, x ) 
    print('\n'+'initial freq point L:', initial_freq_params_L)
    optimal_result = optimizer.optimize(num_vars = 2*p,
                                        objective_function = obj_expectation,
                                        variable_bounds = variable_bounds, 
                                        initial_point = initial_freq_params_L)
    optimal_params_L, exp_val_L, no_evals = optimal_result
    
    if p == 1:
        optimal_params_B, exp_val_B = optimal_params_L.copy(), exp_val_L.copy()
    
    else:
        perturbed_params = [optimal_params_L]
        perturbed_exp_vals = [exp_val_L]
        for l in range(m):
            print('\n'+f'{l}, initial freq point B: ', initial_freq_params_B[l])
            perturbed_optimal_result = optimizer.optimize(num_vars = 2*p,
                                    objective_function = obj_expectation,
                                    variable_bounds= variable_bounds, 
                                    initial_point=initial_freq_params_B[l])
            print('\nNelderMeadcomplete')
            perturbed_optimal_params, perturbed_optimal_expval = perturbed_optimal_result[0:2] 
            perturbed_params.append(perturbed_optimal_params)
            perturbed_exp_vals.append(perturbed_optimal_expval)
        exp_val_B = np.amin(perturbed_exp_vals)
        minim_index = np.where(perturbed_exp_vals == exp_val_B)[0][0]
        optimal_params_B = perturbed_params[minim_index]
    
    return optimal_params_L, optimal_params_B


#Construct QAOA circuit of size p given operator from qubo problem ( operator is obtained from QuadraticProblem.to_ising() )
def construct_QAOAcirc( p, operator ):
    #Initialise parameter list
    params_expr = []
    
    #Create arbitrary "gamma_n" parameters
    for a in range(p):
        params_expr.append( Parameter('a'+str(a+1)) )
    #Likewise for "beta_n" parameters
    for b in range(p):
        params_expr.append( Parameter('b'+str(b+1)) )
        
    #Create circuit given operator, gamma, and beta parameters
    qaoa = QAOA(operator = operator, p=p, quantum_instance=Aer.get_backend('qasm_simulator'))
    circuit = qaoa.construct_circuit( params_expr[0:2*p] )[0]
    return p, params_expr, circuit


#Find expectation value, given qaoa_circuit, qubo_problem and freq_params (u1, u2, ... , v1, v2, ...)
def expectation( qaoa_circuit, qubo_problem, freq_params ):
    simulator = StatevectorSimulator(method = "statevector")
    #unload p, params_expr, circuit from the return value of contruct_QAOAcirc
    p = int(qaoa_circuit[0])
    params_expr = qaoa_circuit[1]
    circuit = qaoa_circuit[2]
    params = convert_fourier_parameters(p,p,freq_params)
    if len(params) != 2*p:
        print("Must have len(params) = 2*no_layers")
    else:    
        circuit2 = circuit.assign_parameters({params_expr[i]: params[i] for i in range(len(params))}, inplace = False)
        result = execute(circuit2, simulator).result()
        statevector = Statevector(result.get_statevector())
        prob_dict = statevector.probabilities_dict(decimals = 6)
        exp_val = 0
        for state in prob_dict:
            exp_val += prob_dict[state] * np.round(eval_cost(state, qubo_problem), 6)
        print(".", end = "")
        return np.round(exp_val, decimals = 5)

#Same as above, but no conversion from fourier space
def expectation_ri( qaoa_circuit, qubo_problem, params ):
    
    #unload p, params_expr, circuit from the return value of contruct_QAOAcirc
    p = int(qaoa_circuit[0])
    params_expr = qaoa_circuit[1]
    circuit = qaoa_circuit[2]
    #params = convert_fourier_parameters(p,p,freq_params)
    if len(params) != 2*p:
        print("Must have len(params) = 2*no_layers")
    else:    
        circuit2 = circuit.assign_parameters({params_expr[i]: params[i] for i in range(len(params))}, inplace = False)
        result = execute(circuit2, simulator).result()
        statevector = Statevector(result.get_statevector())
        prob_dict = statevector.probabilities_dict(decimals = 6)
        exp_val = 0
        for state in prob_dict:
            exp_val += prob_dict[state] * np.round(eval_cost(state, qubo_problem), 6)
        #print(".", end = "")
        return np.round(exp_val, decimals = 5)
    
#Convert frequency parameters to standard gamma and beta parameters
def convert_fourier_parameters(p , q, frequency_parameters):
    u_params = np.array(frequency_parameters[0:q], dtype = float)
    v_params = np.array(frequency_parameters[q:2*q], dtype = float)
    gamma_params = np.empty((p,), dtype = np.float32)
    beta_params = np.empty((p,), dtype = np.float32)
    for i in range(p):
        gamma_inter, beta_inter = 0,0
        a = (i+0.5)*np.pi/p
        for k in range(q):
            gamma_inter += u_params[k]*np.sin( (k+0.5)*a )
            beta_inter += v_params[k]*np.cos( (k+0.5)*a )
        gamma_params[i] = gamma_inter
        beta_params[i] = beta_inter
    return np.append(gamma_params, beta_params)
    
#Binary string x of length 9, using cost/obj function from QUBO on QuadraticProgram qp.
def eval_cost(x, qubo_problem):
    y = [int(x[no_qubits-1-i]) for i in range(no_qubits)] #reverse order
    value = qubo_problem.objective.evaluate(y)
    return value

#Function make a QAOA circuit given no of layers, fourier parameters, and the operator obtained from QUBO
def parametrise_QAOAcirc( p, fourier_params, operator ):
    p, params_expr, circuit = construct_QAOAcirc(p, operator)
    params = convert_fourier_parameters(p,p, fourier_params)
    circuit2 = circuit.assign_parameters({params_expr[i]: params[i] for i in range(len(params))}, inplace = False)
    circuit2.measure_active()
    return circuit2    

In [15]:
#Print first 20 lowest states according to QUBO cost function, 
#Prints the binary string, then the cost, and "True" if it is a valid solution, i.e. if it satisfies all constraints

values = {}
for k in range(2**9):
    string = '{0:09b}'.format(k)
    values[string] = eval_cost(string, qubo)
sort_values = sorted(values.items(), key=lambda x: x[1])
i=0

#Save costs list to a file
costs = open("costs.txt", "w+")
for string, cost in sort_values:
    print(string, round(cost,3), string[0:3].count('1')==1 and string[3:6].count('1')==1 and string[6:9].count('1')==1)
    costs.writelines(L for L in [string+" ", str(round(cost,3))+" ", str( string[0:3].count('1')==1 and string[3:6].count('1')==1 and string[6:9].count('1')==1 )+" ", "\n"])
    i += 1
    if i == 20:
        break
costs.close()

001010010 1.3 True
001001010 1.364 True
100001010 1.502 True
001010001 1.513 True
010001010 1.534 True
100010010 1.545 True
010010001 1.556 True
010010010 1.566 True
000001010 1.568 False
001000010 1.568 False
001100010 1.577 True
001001100 1.588 True
010001100 1.63 True
100010001 1.63 True
001010100 1.63 True
100001100 1.705 True
000010001 1.718 False
001010000 1.718 False
010010100 1.769 True
000001100 1.792 False


In [16]:
#Now Optimise up to {k} layers, and at each layer, generate {no_perturb} number of random points based on Lukin's Fourier Space method
#E.g. for k=2 layers, with 3 random points, (The solution converges well for up to ~20 layers, with around 10 random points)
k=2
no_perturb = 3

#Note the output prints a dot each time the expectation function is called, so it will print many as it computes lots of expectation values. This way I know the code is still running

#Use lists to append results from optimization
optimal_params_L_s = []
optimal_params_B_s = []
for i in range(k):
    if i==0:
        optimal_params_L, optimal_params_B = optimize_qubo(qubo, i+1, [], [], no_perturb)
    else:
        optimal_params_L, optimal_params_B = optimize_qubo(qubo, i+1, optimal_params_L_s[i-1], optimal_params_B_s[i-1], no_perturb)
    optimal_params_L_s.append(optimal_params_L)
    optimal_params_B_s.append(optimal_params_B)
    
    #At each layer, save results to pkl file
    with open('optimal_params_L_s_wonoise.pkl', 'wb') as f:
        pickle.dump(optimal_params_L_s, f)
    with open('optimal_params_B_s_wonoise.pkl', 'wb') as f:
        pickle.dump(optimal_params_B_s, f)    


Step 1: Building circuit...

Step 1: Now optimising parameters (Nelder-Mead)...

initial freq point L: [0.05 0.05]
............

KeyboardInterrupt: 